# Tutorial: Generating Structured Output with OpenAIChatGenerator

- **Level**: Beginner
- **Time to complete**: 15 minutes
- **Prerequisites**: You must have an API key from an active OpenAI account as this tutorial is using the gpt-4o-mini model by OpenAI.
- **Components Used**: `ChatPromptBuilder`, `OpenAIChatGenerator`, `OpenAIResponsesChatGenerator`
- **Goal**: Learn how to generate structured outputs with `OpenAIChatGenerator` or `OpenAIResponsesChatGenerator` using Pydantic model or JSON schema.

## Overview
This tutorial shows how to produce structured outputs by either providing [Pydantic](https://github.com/pydantic/pydantic) model or JSON schema to `OpenAIChatGenerator`.

Note: Only latest model starting with `gpt-4o-mini` can be used for this feature.


## Installing Dependencies
Install Haystack and [colorama](https://pypi.org/project/colorama/) with pip:

In [9]:
%%bash

pip install "haystack-ai==2.20.0"
pip install colorama

## Structured Outputs with `OpenAIChatGenerator`

### Using Pydantic Models
First, we'll see how to pass Pydantic model to `OpenAIChatGenerator`. For this purpose, we define two [Pydantic models](https://docs.pydantic.dev/1.10/usage/models/), `City` and `CitiesData`. These models specify the fields and types that represent the data structure we want.

In [10]:
from typing import List
from pydantic import BaseModel


class City(BaseModel):
    name: str
    country: str
    population: int


class CitiesData(BaseModel):
    cities: List[City]

> You can change these models according to the format you wish to extract from the text.


Use `ChatPromptBuilder` in the pipeline to pass the user’s message to `OpenAIChatGenerator`.
For information about Jinja2 template and ChatPromptBuilder, see [ChatPromptBuilder](https://docs.haystack.deepset.ai/docs/chatpromptbuilder).

In [11]:
from haystack.components.builders import ChatPromptBuilder
from haystack.dataclasses import ChatMessage


prompt_template = [ChatMessage.from_user("User Input: {{passage}}")]
prompt_builder = ChatPromptBuilder(template=prompt_template)


[OpenAIChatGenerator](https://docs.haystack.deepset.ai/docs/openaichatgenerator) generates
text using OpenAI's `gpt-4o-mini` model by default. We pass our Pydantic model to `response_format` paramter in generation_kwargs .

We also need to set the `OPENAI_API_KEY` variable.

Note: You can also set the `response_format` in `generation_kwargs` param in the `run` method of chat generator.

In [12]:
import os
from getpass import getpass

from haystack.components.generators.chat import OpenAIChatGenerator

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter OpenAI API key:")
chat_generator = OpenAIChatGenerator(generation_kwargs={"response_format": CitiesData})


Add all components to your pipeline and connect them.

In [13]:
from haystack import Pipeline

pipeline = Pipeline(max_runs_per_component=5)

# Add components to your pipeline
pipeline.add_component(instance=prompt_builder, name="prompt_builder")
pipeline.add_component(instance=chat_generator, name="llm")

# Now, connect the components to each other
pipeline.connect("prompt_builder.prompt", "llm.messages")


🚅 Components
  - prompt_builder: ChatPromptBuilder
  - llm: OpenAIChatGenerator
🛤️ Connections
  - prompt_builder.prompt -> llm.messages (list[ChatMessage])

## Testing the Pipeline

Run the pipeline with an example passage that you want to convert into a JSON format and the `json_schema` you have created for `CitiesData`. For the given example passage, the generated JSON object should be like:
```json
{
  "cities": [
    {
      "name": "Berlin",
      "country": "Germany",
      "population": 3850809
    },
    {
      "name": "Paris",
      "country": "France",
      "population": 2161000
    },
    {
      "name": "Lisbon",
      "country": "Portugal",
      "population": 504718
    }
  ]
}
```
The output of the LLM should be compliant with the `json_schema`. 

In [14]:
passage = "Berlin is the capital of Germany. It has a population of 3,850,809. Paris, France's capital, has 2.161 million residents. Lisbon is the capital and the largest city of Portugal with the population of 504,718."
result = pipeline.run({"prompt_builder": {"passage": passage}})

### Print the Correct JSON
If you didn't get any error, you can now print the corrected JSON.

In [15]:
import json
valid_reply = result["llm"]["replies"][0].text
valid_json = json.loads(valid_reply)
print(valid_json)

{'cities': [{'name': 'Berlin', 'country': 'Germany', 'population': 3850809}, {'name': 'Paris', 'country': 'France', 'population': 2161000}, {'name': 'Lisbon', 'country': 'Portugal', 'population': 504718}]}


### Using Json schema

Now, we’ll create a json schema of the `CitiesData` model and pass it to `OpenAIChatGenerator`. OpenAI expects schemas in a specific format, so the schema generated with `model_json_schema()` cannot be used directly.

For details on how to create schemas for OpenAI, see the [OpenAI Structured Outputs guide](https://platform.openai.com/docs/guides/structured-outputs#supported-schemas).

In [16]:
cities_data_schema={
        "type": "json_schema",
        "json_schema": {
            "name": "CitiesData",
            "schema": {
                "type": "object",
                "properties": {
                    "cities": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "name": { "type": "string" },
                                "country": { "type": "string" },
                                "population": { "type": "integer" }
                            },
                            "required": ["name", "country", "population"],
                            "additionalProperties": False
                        }
                    }
                },
                "required": ["cities"],
                "additionalProperties": False
            },
            "strict": True
        }
    }

Pass this json schema to the `response_format` parameter in chat generator. We run the generator indivdually to see the output.

In [17]:
chat_generator = OpenAIChatGenerator(generation_kwargs={"response_format": cities_data_schema})

messages = [ChatMessage.from_user("Berlin is the capital of Germany. It has a population of 3,850,809. Paris, France's capital, has 2.161 million residents. Lisbon is the capital and the largest city of Portugal with the population of 504,718.")]

result = chat_generator.run(messages=messages)

print(result["replies"][0].text)

{"cities":[{"name":"Berlin","country":"Germany","population":3850809},{"name":"Paris","country":"France","population":2161000},{"name":"Lisbon","country":"Portugal","population":504718}]}


## Structured Outputs with `OpenAIResponsesChatGenerator`

### Using Pydantic Models
We'll use the models City and CitiesData defined above. 
[OpenAIResponsesChatGenerator](https://docs.haystack.deepset.ai/docs/openairesponseschatgenerator) generates
text using OpenAI's `gpt-5-mini` model by default. We pass our Pydantic model to `text_format` parameter in `generation_kwargs` when calling the `run` method.

We also need to set the `OPENAI_API_KEY` variable.

Note: You can also set the `text_format` in the constructor's `generation_kwargs`, but passing it in the `run` method is recommended for better compatibility.


In [18]:
import os
from getpass import getpass

from haystack.components.generators.chat import OpenAIResponsesChatGenerator

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter OpenAI API key:")
responses_generator = OpenAIResponsesChatGenerator(generation_kwargs={"text_format": CitiesData})

Lets check the structured output with a simple user message.

In [19]:
responses_generator.run(messages=[ChatMessage.from_user(" Berlin is the capital of Germany. It has a population of 3,850,809. Paris, France's capital, has 2.161 million residents.")])

{'replies': [ChatMessage(_role=<ChatRole.ASSISTANT: 'assistant'>, _content=[ReasoningContent(reasoning_text='', extra={'id': 'rs_02308545114ab79f00691dee6614f08198a6a7a20c09e0071e', 'type': 'reasoning'}), TextContent(text='{\n  "cities": [\n    {\n      "name": "Berlin",\n      "country": "Germany",\n      "population": 3850809\n    },\n    {\n      "name": "Paris",\n      "country": "France",\n      "population": 2161000\n    }\n  ]\n}')], _name=None, _meta={'id': 'resp_02308545114ab79f00691dee6528d08198bf3e9e82903570a4', 'created_at': 1763569253.0, 'error': None, 'incomplete_details': None, 'instructions': None, 'metadata': {}, 'model': 'gpt-5-mini-2025-08-07', 'object': 'response', 'parallel_tool_calls': True, 'temperature': 1.0, 'tool_choice': 'auto', 'tools': [], 'top_p': 1.0, 'background': False, 'max_output_tokens': None, 'max_tool_calls': None, 'previous_response_id': None, 'prompt_cache_key': None, 'reasoning': {'effort': 'medium', 'summary': None}, 'safety_identifier': None, 

Now, we’ll create a json schema of the `CitiesData` model and pass it to `OpenAIResponsesChatGenerator`. We cannot use the same schema we defined for `OpenAIChatGenerator` as OpenAI Responses API expects a different format of schema.


In [21]:
cities_data_schema_responses={
    "format": {
        "type": "json_schema",
            "name": "CitiesData",
            "schema": {
                "type": "object",
                "properties": {
                    "cities": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "name": { "type": "string" },
                                "country": { "type": "string" },
                                "population": { "type": "integer" }
                            },
                            "required": ["name", "country", "population"],
                            "additionalProperties": False
                        }
                    }
                },
                "required": ["cities"],
                "additionalProperties": False
            },
            "strict": True
        }
    }

We pass our json schema to `text` paramter in `generation_kwargs`.

Note: You can also set the `text` in `generation_kwargs` param in the `run` method of the chat generator.

In [23]:
chat_generator = OpenAIResponsesChatGenerator(generation_kwargs={"text": cities_data_schema_responses})

result = chat_generator.run(messages=[ChatMessage.from_user(" Berlin is the capital of Germany. It has a population of 3,850,809. Paris, France's capital, has 2.161 million residents.")])
parsed = json.loads(result["replies"][0].text)

print(parsed)

{'cities': [{'name': 'Berlin', 'country': 'Germany', 'population': 3850809}, {'name': 'Paris', 'country': 'France', 'population': 2161000}]}


## What's next

🎉 Congratulations! You've learnt how to easily produce structured ouputs with `OpenAIChatGenerator` and `OpenAIResponsesChatGenerator` using Pydantic models and JSON schema.

Other chat generators that also support structured outputs: [`MistralChatGenerator`](https://docs.haystack.deepset.ai/docs/mistralchatgenerator), [`OpenRouterChatGenerator`](https://docs.haystack.deepset.ai/docs/openrouterchatgenerator), [`NvidiaChatGenerator`](https://docs.haystack.deepset.ai/docs/nvidiachatgenerator), [`MetaLlamaChatGenerator`](https://docs.haystack.deepset.ai/docs/metallamachatgenerator), [`TogetherAIChatGenerator`](https://docs.haystack.deepset.ai/docs/togetheraichatgenerator),
[`LlamaStackChatGenerator`](https://docs.haystack.deepset.ai/docs/llamastackchatgenerator) and [`STACKITChatGenerator`](https://docs.haystack.deepset.ai/docs/stackitchatgenerator).

To stay up to date on the latest Haystack developments, you can [subscribe to our newsletter](https://landing.deepset.ai/haystack-community-updates) and [join Haystack discord community](https://discord.gg/haystack).

Thanks for reading!